In [1]:
from typing import List, Optional
from langchain_core.embeddings import Embeddings
import requests

class DeepSeekEmbeddings(Embeddings):
    def __init__(self, api_key: str, model: str = "deepseek-embedding"):
        self.api_key = api_key
        self.model = model
        self.endpoint = "https://api.deepseek.com/v1/embeddings"

    def _embed(self, text: str) -> List[float]:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        data = {
            "model": self.model,
            "input": text
        }
        response = requests.post(self.endpoint, headers=headers, json=data)
        response.raise_for_status()  # 检查HTTP错误
        return response.json()['data'][0]['embedding']

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """批量文本生成 Embedding"""
        return [self._embed(text) for text in texts]

    def embed_query(self, text: str) -> List[float]:
        """单个查询文本生成 Embedding"""
        return self._embed(text)

In [2]:
from langchain_community.vectorstores import FAISS

# 初始化 Embedding 对象
embeddings = DeepSeekEmbeddings(api_key="YOUR_API_KEY")

# 创建向量数据库
texts = ["DeepSeek 是中国的 AGI 公司", "Meta 总部位于美国"]
vector_db = FAISS.from_texts(texts, embeddings)

# 语义搜索
query = "中国的AGI企业"
docs = vector_db.similarity_search(query, k=1)
print(docs[0].page_content)  # 输出: DeepSeek 是中国的 AGI 公司

HTTPError: 404 Client Error: Not Found for url: https://api.deepseek.com/v1/embeddings